<div style="color: #2590c2; text-align: center;">
<span style="font-size:18pt;"><b>ST: DEEP LEARNING</b></span><br/>
<span>(CS 696-04) (SM18)</span><br/><br/>
<span><b>Homework 3</b></span><br/><br/>
<span>Submitted By</span><br/>
<span>Ashok Kumar Shrestha</span>
</div>

<h4><u>Tasks:</u></h4><br/>
<span>In homework 3, you are asked to implement either your homework 1 or your homework 2 using tensorflow, keras or PyTorch.<span><br/>
<span>Please also submit a report on Wednesday in class. </span>

<h4>Implementation:</h4><br/>
<span>Implementing Homework-2 in Keras</span><br/>
<h4>Testing results:</h4><br/>
<div style="width: 700px;height:140px;">
    <div style="float: left;">
        <b>Data: CIFAR10</b><br/>
        batch_size = 512<br/>
        epochs = 15<br/>
        Train loss: 0.5120<br/>
        Train accuracy: 82.75 %<br/>
        Test loss: 0.9040<br/>
        Test accuracy: 70.55 %<br/>
    </div>
    <div style="float: right;">
        <b>Data: MNIST</b><br/>
        batch_size = 512<br/>
        epochs = 2<br/>
        Train loss: 0.0638<br/>
        Train accuracy: 98.09 %<br/>
        Test loss: 0.0416<br/>
        Test accuracy: 98.67 %
    </div>
    
</div>
<h4> CIFAR10:</h4>
<div style="width: 800px;height:200px;">
    <img src="img/cost_vs_epochs.png" style="height: 200px;float: left;"/>
    <img src="img/accuracy_vs_epochs.png" style="height: 200px;float: right;"/>
</div>


<b>
Code:
</b><br/>
<span>
Read data sets (MNIST and CIFAR-10) from the file.
</span>

In [ ]:
"""
Read Data sets: MNIST and CIFAR-10
-----------------------------------------------
Parameters:
===========
file_name: file name to read

Return:
=======
train_img, test_img, train_lbl, test_lbl values
"""
import cloudpickle as pickle
from sklearn.model_selection import train_test_split
import _pickle as pickle
import numpy as np
import os

def get_CIFAR10_data(cifar10_dir, num_training=49000, num_validation=1000, num_test=1000):
    # Load the raw CIFAR-10 data
    X_train, y_train, X_test, y_test = load(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    X_train = X_train.astype(np.float64)
    X_val = X_val.astype(np.float64)
    X_test = X_test.astype(np.float64)

    # Transpose so that channels come first
    X_train = X_train.transpose(0, 3, 1, 2)
    X_val = X_val.transpose(0, 3, 1, 2)
    X_test = X_test.transpose(0, 3, 1, 2)
    mean_image = np.mean(X_train, axis=0)
    std = np.std(X_train)

    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    X_train /= std
    X_val /= std
    X_test /= std
    
    return {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test,
        'mean': mean_image, 'std': std
    }

def load_CIFAR_batch(filename):
    ''' load single batch of cifar '''
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding ='bytes')
        X = datadict[b'data']
        Y = datadict[b'labels']
        X = X.reshape(10000, 3, 32, 32)
        Y = np.array(Y)
        return X, Y

def get_CIFAR10(ROOT):
    ''' load all of cifar '''
    xs = []
    ys = []
    for b in range(1, 6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Xte, Ytr, Yte

def load_mnist(data_file="mnist.data", test_size=0.10, random_state=0):
    mnist = pickle.load(open(data_file, "rb"))
    
    mnist['data'] = np.reshape(mnist['data'],(mnist['data'].shape[0],1,28,28))
    return train_test_split(mnist['data'], mnist['target'], test_size=test_size,
                            random_state=random_state)

def load(file_name):
    if file_name == "mnist":
        return load_mnist(data_file="mnist.data", test_size=0.2, random_state=42)
    
    elif file_name == "cifar10":
        return get_CIFAR10("")

<b>Main Program:</b>

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.utils import np_utils
from keras.datasets import mnist
from keras.datasets import cifar10
from keras import backend as K
K.set_image_dim_ordering('th')
import matplotlib.pyplot as plt

def show_graph(epoch, history, title="Cost vs Epoch",xlabel="Epochs",ylabel="Training Cost",image_name="cost_vs_epochs.png"):
        epochs = range(epoch)
        
        epochs = np.reshape(epochs,(-1,1))
        history = np.reshape(history,(-1,1)) *100
        
        plt.plot(epochs, history,"r--")
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.savefig(image_name)
        plt.show()

#Read data from Keras' datasets
def read_data_keras(file="mnist"):
    if file=="mnist":
        (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
        c = 1
        w,h = X_train.shape[1:]
        print("MNIST data loaded.")
    elif file=="cifar10":
        (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
        c,w,h = X_train.shape[1:]
        print("CIFAR data loaded.")
    
    X_train = X_train.reshape(X_train.shape[0], c,w,h)
    X_test = X_test.reshape(X_test.shape[0], c,w,h)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255

    Y_train = np_utils.to_categorical(Y_train, 10)
    Y_test = np_utils.to_categorical(Y_test, 10)
    
    return (X_train, Y_train), (X_test, Y_test)

#Read data from dataset files provided
def read_data_files(file="mnist"):
    if file=="mnist":
        X_train, X_test, Y_train, Y_test = load(file_name=file)
        print("MNIST data loaded.")
    elif file=="cifar10":
        X_train, X_test, Y_train, Y_test = load(file_name=file)
        print("CIFAR data loaded.")
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255

    Y_train = np_utils.to_categorical(Y_train, 10)
    Y_test = np_utils.to_categorical(Y_test, 10)
    
    return (X_train, Y_train), (X_test, Y_test)

def main(file="mnist", batch_size = 512, epochs = 10):
    print("Starting Network...")
    print("-------------------------------------------------------")
    print("Reading Data sets...")
    
    #(X_train, Y_train), (X_test, Y_test) = read_data_keras(file)
    (X_train, Y_train), (X_test, Y_test) = read_data_files(file)
    
    input_shape = X_train.shape[1:]
    num_classes = 10
    
    print("-------------------------------------------------------")
    print("Begin Training...")
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    history = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)
    
    print("End Training.")
    print("-------------------------------------------------------")
    print("Begin Testing...")
    
    score = model.evaluate(X_test, Y_test, verbose=0)
    test_loss = score[0]
    test_accuracy = score[1]*100
    
    print("End Testing.")
    print("-------------------------------------------------------")
    
    print('Training Accuracy: {0}'.format(test_loss))
    print('Test Accuracy: {0:0.2f} %'.format(test_accuracy))
    
    show_graph(epochs, history.history['loss'], 
               title="Cost vs Epoch",
               xlabel="Epochs",
               ylabel="Training Cost",
               image_name="cost_vs_epochs.png")
    
    show_graph(epochs, history.history['acc'], 
               title="Accuracy vs Epoch",
               xlabel="Epochs",
               ylabel="Accuracy",
               image_name="accuracy_vs_epochs.png")


if __name__=="__main__":
    main(file="cifar10", batch_size = 512, epochs = 15)
    